In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req =  requests.get(url)
html = req.text



In [3]:
soup = BeautifulSoup(html, 'html.parser')

In [4]:
bse = soup.select('div.rankBestContentList > ol > li')
bse

[<li>
 <!-- listItem | singleType or setType :: start -->
 <div class="listItem singleType">
 <div class="cover">
 <div class="coverImage">
 <label>
 <a href="/product/BookDisplay.do?_method=detail&amp;sc.shopNo=0000400000&amp;sc.prdNo=354069935&amp;sc.saNo=003003001&amp;bid1=Best_zone&amp;bid2=LiveRanking&amp;bid3=PRD&amp;bid4=001" target="_blank">
 <span class="checkBox">
 <input name="_KEY" type="checkbox" value="354069935"/>
 </span>
 <img alt="트렌드 코리아 2022" src="https://bimage.interpark.com/partner/goods_image/9/9/3/5/354069935s.jpg"/>
 <i class="decoFront"></i>
 </a>
 </label>
 </div>
 <div class="coverOption">
 <ul>
 <li class="cart">
 <a class="cartGo" href="#" value="354069935"><span>CART</span></a>
 </li>
 <li class="buy">
 <a href="javascript:directOrd('354069935');"><span>BUY</span></a>
 </li>
 </ul>
 </div>
 <div class="rankNumber digit2">
 <span class="rankBtn_ctrl rkNum_B01"></span>
 </div>
 </div>
 <div class="itemIcon">
 <span class="bestSeller">베스트셀러</span>
 </div>
 <

In [5]:
bs = bse[11]


In [6]:
#순위
#rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:]
rank = bs.select('.rankNumber.digit2>span')
if len(rank) == 1 :
    rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:]
else:
    rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:] + bs.select_one('.rankNumber.digit2').find_all('span')[1]['class'][1][-1:]
rank

'12'

In [7]:
#title
title = bs.select_one('div.itemName').get_text()
title

'소크라테스 익스프레스(20만 부 기념 윈터 에디션) '

In [8]:
#가격
price = bs.select_one('div.priceWrap').get_text().split('\n')[2:4]
price = int(price[0].replace(',',''))
price


16200

In [9]:
bs = bse[0]
href = bs.find('a')['href'][69:78]
href

'354069935'

In [10]:
sub_url = f'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo={href}&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [11]:
sub_req =  requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')


In [12]:
#저자
writer = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[1][4:]
writer


'김난도, 전미영, 최지혜, 이향은, 이준영'

In [13]:
#출판사
company = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[2][6:]
company

'미래의창'

In [14]:
# 발행일
year = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[4][5:]
year

'2021년 10월 06일'

In [15]:
# 쪽수
page = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[-4][5:]
page

'452'

In [16]:
lines = []
for bs in bse:
    href = bs.find('a')['href'][69:78]
    rank = bs.select('.rankNumber.digit2>span')
    if len(rank) == 1 :
        rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:]
    else:
        rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:] + bs.select_one('.rankNumber.digit2').find_all('span')[1]['class'][1][-1:]
    title = bs.select_one('div.itemName').get_text()
    price = bs.select_one('div.priceWrap').get_text().split('\n')[2:4]
    price = int(price[0].replace(',',''))
    price

    sub_url = f'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo={href}&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'
    sub_req =  requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')
    writer = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[1][4:]
    company = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[2][6:]
    year = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[4][5:]
    page = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[-4][5:]

    lines.append([rank,title,price,writer,company,year,page])
    

In [17]:
pd.DataFrame(lines, columns=['순위','제목','가격','저자','출판사','발행일','쪽수'])

,순위,제목,가격,저자,출판사,발행일,쪽수
0,1,트렌드 코리아 2022,16200,"김난도, 전미영, 최지혜, 이향은, 이준영",미래의창,2021년 10월 06일,452
1,2,2022 현직교사들이 들려주는 면접레시피,42300,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,2021년 11월 12일,1104
2,3,거인의 포트폴리오,16200,강환국,페이지2북스,2021년 11월 08일,404
3,3,거꾸로 읽는 세계사,15750,유시민,돌베개,2021년 10월 29일,404
4,5,아직 오지 않은 날들을 위하여,14400,파스칼 브뤼크네르역 : 이세진,인플루엔셜,2021년 11월 12일,320
5,6,오십에 읽는 논어,14400,최종엽,유노북스,2021년 11월 03일,300
6,7,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),13500,소윤,북로망스,2021년 09월 29일,274
7,8,투자는 디테일에 있다,16200,김정환,부케이,2021년 12월 10일,424
8,8,웰씽킹(Wealthinking),14400,켈리 최,다산북스,2021년 11월 10일,316
9,8,달러구트 꿈 백화점. 2,12420,이미예,팩토리나인,2021년 07월 27일,308
